In [ ]:

# ------------------------------------------------------------------
# Load Libraries & Data
# ------------------------------------------------------------------
import pandas as pd
import numpy as np
import os
import re
import importlib
import f_data_processing as dp
dp = importlib.reload(dp)
print(os.getcwd())

['map_cols', 'rename_columns', 'to_num_col']
c:\Users\stdt\ETH Zurich\PhD Veronika - 07_Master Thesis Natalie\10_hcm_code\code


In [ ]:
# latent constructs:
    # likert: sensitivity_nh_1-3, climatechange_nh_1, finan_vulnerability_1, cost_cc_policy_1, psycho_distance1-4
    # choice set: experience_nh
    
# choice experiment
    # 1-9_conjoint_acceptance_1&2

valid_columns_s1 = [
    "sensitivity_nh_1", "sensitivity_nh_2", "sensitivity_nh_3", "sensitivity_nh_4",
    "costs_cc_policy_1", "finan_vulnerability_1",
    "climatechange_nh_1", "climatechange_nh_2", "climatechange_nh_3",
    "psycho_distance_1", "psycho_distance_2", "psycho_distance_3", "psycho_distance_4",
    "experience_nh",
    "number_household _1_TEXT",
    "1_conjoint_acceptance_1", "1_conjoint_acceptance_2",
    "2_conjoint_acceptance_1", "2_conjoint_acceptance_2",
    "3_conjoint_acceptance_1", "3_conjoint_acceptance_2",
    "4_conjoint_acceptance_1", "4_conjoint_acceptance_2",
    "5_conjoint_acceptance_1", "5_conjoint_acceptance_2",
    "6_conjoint_acceptance_1", "6_conjoint_acceptance_2",
    "7_conjoint_acceptance_1", "7_conjoint_acceptance_2",
    "8_conjoint_acceptance_1", "8_conjoint_acceptance_2",
    "9_conjoint_acceptance_1", "9_conjoint_acceptance_2",
    "own_municipality_1", "own_municipality_2", "own_municipality_3", "own_municipality_4",
    "gal_tan_1", "gal_tan_2", "gal_tan_3", "gal_tan_4",
    "lreco_1", "lreco_2", "lreco_3",
    "env_values_1", "env_values_2", "env_values_3",
    "deservingness_1", "deservingness_2", "deservingness_3", "deservingness_4", "deservingness_5", "deservingness_6",
    "deservingness_7", "deservingness_8", "deservingness_9", "deservingness_10", "deservingness_11", "deservingness_12"    
]

likert_map = {
    "1 - Extremely unlikely": 1,
    "1 - Strongly disagree": 1,
    "1 - Totally unacceptable": 1,
    "1 - Very unlikely": 1,
    "1 - Very difficult": 1,
    "6 - Totally acceptable": 6,
    "6 - Strongly agree": 6,
    "6 - Very likely": 6,
    "6 - Very easy": 6,
    "6 - Extremely worried": 6,
    "6 - Extremely likely": 6,
    "I don't know": np.nan,
    "Prefer not to say": np.nan,
    "I don't now": np.nan,
    "7": np.nan
}


demographics_dict = {
    # gender
    "Female": "Female",
    "Male": "Male",
    "Non-binary / Other": np.nan,
    "Prefer not to say": np.nan,  # beachte Leerzeichen wie im R-Code
    # age
    "18 - 34": "18 - 34",
    "35 - 49": "35 - 49",
    "50 or older": "50+",
    # education
    "No high school diploma": "Below Secondary",
    "Vocational training or apprenticeship": "Vocational training or apprenticeship",
    "High school diploma": "High school diploma",
    "Bachelor's degree": "University degree",
    "Master's degree": "University degree",
    "Doctoral or professional degree (e.g. PhD, MD, JD)": "University degree",
    "Prefer not to say": np.nan,
    # income
    # TODO: what source?
    "Less than CHF 50,000": "Low",
    "CHF 50,000 - CHF 70,000": "Low",
    "CHF 70,000 - CHF 100,000": "Mid",
    "CHF 100,001 - CHF 150,000": "Mid",
    "CHF 150,001 - CHF 250,000": "High",
    "More than CHF 250,000": "High",
    "I don't know": np.nan,
    "Prefer not to say": np.nan,
    # language
    "Deutsch": "German",
    "Français": "French",
    "English": "English",
    "Italiano": "Italian",
    # region
    "German-speaking region": "German-speaking region",
    "Italian-speaking region": "Italian-speaking region",
    "French-speaking region": "French-speaking region",
    "Romansh-speaking region": "Romansh-speaking region",
    # political
    "Social Democratic Party (SP)": "Left",
    "The Greens (GPS)": "Left",
    "Green Liberals (GLP)": "Liberal",
    "The Liberals (FDP)": "Liberal",
    "The Middle Party (merger between the Christian Democratic People's Party (CVP) and the Civic Democratic Party (BDP))": "Liberal",
    "Swiss People's Party (SVP)": "Conservative",
    "Federal Democratic Union (EDU)": "Conservative",
    "Evangelical People's Party of Switzerland (EPP)": "Conservative",
    "Mouvement Citoyens Genevois (MCG)": "Conservative",
    "Lega dei Ticinesi (Lega)": "Conservative",
    "Others, such as": np.nan,
    "I don't feel close to any party": "No party association",
    "Prefer not to say": np.nan,
}

translation_dict = {
    # Costs
    "Tous les citoyens paient le même montant": "All people pay the same amount",
    "Alle Menschen zahlen den gleichen Betrag": "All people pay the same amount",
    "Tutte le persone pagano lo stesso importo": "All people pay the same amount",

    "Les personnes paient proportionnellement à leurs revenus": "People pay proportionally to their income",
    "Menschen zahlen proportional zu ihrem Einkommen": "People pay proportionally to their income",
    "Le persone pagano in proporzione al loro reddito": "People pay proportionally to their income",

    "Les personnes et entreprises bénéficiant des mesures de protection": "People & companies being protected by protective measures",
    "Menschen und Unternehmen, die von Schutzmaßnahmen profitieren": "People & companies being protected by protective measures",
    "Le persone e aziende che beneficiano di misure di protezione": "People & companies being protected by protective measures",

    "Les personnes paient proportionnellement à leurs émissions de CO2": "People pay proportionally to their CO2 emissions",
    "Menschen zahlen proportional zu ihrem CO2-Ausstoss": "People pay proportionally to their CO2 emissions",
    "Le persone pagano in proporzione alle loro emissioni di CO2": "People pay proportionally to their CO2 emissions",

    "Les entreprises paient proportionnellement à leurs émissions de CO2": "Companies pay proportionally to their CO2 emissions",
    "Unternehmen zahlen proportional zu ihrem CO2-Ausstoss": "Companies pay proportionally to their CO2 emissions",
    "Le aziende pagano in proporzione alle loro emissioni di CO2": "Companies pay proportionally to their CO2 emissions",

    # Exemptions
    "Les personnes à faible revenu peuvent être exemptées des coûts": "Low-income earners exempted from costs",
    "mit Ausnahme von Menschen mit niedrigem Einkommen": "Low-income earners exempted from costs",
    "Le persone a basso reddito sono esentate dai costi": "Low-income earners exempted from costs",

    "Les personnes à faibles et moyens revenus peuvent être exemptées des coûts": "Low- and middle-income earners exempted from costs",
    "mit Ausnahme von Menschen mit niedrigem und mittlerem Einkommen": "Low- and middle-income earners exempted from costs",
    "Le persone a basso e medio reddito sono esentate dai costi": "Low- and middle-income earners exempted from costs",

    "Aucun groupe n'est exempté des coûts": "No groups exempted from costs",
    "Keine Gruppen sind von den Kosten ausgenommen": "No groups exempted from costs",
    "Nessun gruppo è esentato dai costi": "No groups exempted from costs",

    # Benefits
    "Les municipalités les plus touchées par les risques naturels, même si elles sont en déclin économique": "Municipalities most affected by natural hazards even if they are economically declining",
    "Gemeinden, die am stärksten von Naturgefahren betroffen sind, selbst wenn sie wirtschaftlich im Rückgang sind": "Municipalities most affected by natural hazards even if they are economically declining",
    "I comuni più a rischio dai pericoli naturali, anche se sono economicamente in declino": "Municipalities most affected by natural hazards even if they are economically declining",

    "Les municipalités économiquement prospères": "Economically prosperous municipalities",
    "Wirtschaftlich wohlhabende Gemeinden": "Economically prosperous municipalities",
    "I comuni economicamente prosperi": "Economically prosperous municipalities",

    "Les communes dans lesquelles les gens vivent depuis de nombreuses années doivent être protégées à tout prix": "Municipalities in which people have lived in for many years should be protected at all costs",
    "Gemeinden, in denen Menschen seit vielen Jahren leben, sollten um jeden Preis geschützt werden": "Municipalities in which people have lived in for many years should be protected at all costs",
    "I comuni in cui le persone vivono da molti anni devono essere protetti ad ogni costo": "Municipalities in which people have lived in for many years should be protected at all costs",

    "Niveaux de protection égaux pour toutes les municipalités": "Equal protection levels for all municipalities",
    "Gleiche Schutzniveaus für alle Gemeinden": "Equal protection levels for all municipalities",
    "Livelli di protezione uguali per tutti i comuni": "Equal protection levels for all municipalities",

    "Municipalités ayant une grande valeur culturelle, par exemple celles dotées de bâtiments historiques": "Culturally valuable municipalities e.g. with historic buildings",
    "Gemeinden mit vielen Kulturgütern wie z.B. historischen Gebäuden": "Culturally valuable municipalities e.g. with historic buildings",
    "Comuni di grande valore culturale, ad esempio con edifici storici": "Culturally valuable municipalities e.g. with historic buildings",
}



In [ ]:
# ------------------------------------------------------------------
# helper functions
# ------------------------------------------------------------------

df = pd.read_csv("../data/Survey_Adaptation Natural Hazards_Second Wave_raw data.csv", dtype={"id": "string"}, skiprows=[1,2])

df = dp.rename_columns(df, 'municipality', 'benefit')

df = df.filter(
    regex=r"^respondent_id$|^choice(?!$)|(?!^)prefer$",
    axis=1
)

df_task = df.drop(columns=df.filter(regex='pay|exemp').columns)



0      318078313787574
1      318078313789657
2      318078313783371
3      318078313788610
4      318078313784432
            ...       
769    318152003806456
770    318078398528518
771    318078398530694
772    318078398528553
773    318152004293638
Name: m, Length: 774, dtype: int64

In [ ]:

# 1) Discrete choice tasks (Attribute pro Option)

# respondent_id + alle Spalten, die mit "choice" anfangen
df_6_distr = df_6[["respondent_id"] + df_6.filter(regex="^choice").columns.tolist()]
df_6_distr = df_6_distr.melt(id_vars="respondent_id", var_name="variable", value_name="value")

df_6_distr["taskNum"]   = df_6_distr["variable"].str.replace("[A-Za-z]|_.+", "", regex=True)
df_6_distr["optionNum"] = df_6_distr["variable"].str[-1].astype(int)
df_6_distr["attribute"] = df_6_distr["variable"].str.replace(".+_|.$", "", regex=True)

df_6_distr = (
    df_6_distr
    .pivot_table(index=["respondent_id", "taskNum", "optionNum"],
                 columns="attribute", values="value", aggfunc="first")
    .reset_index()
)
df_6_distr.columns.name = None


# 2) Präferenzen je Task

df_6_prefer = df_6[["respondent_id"] + df_6.filter(regex="prefer$").columns.tolist()]
df_6_prefer = df_6_prefer.melt(id_vars="respondent_id", var_name="variable", value_name="preference")

df_6_prefer["taskNum"] = df_6_prefer["variable"].str.replace("^X|_conjoint_prefer", "", regex=True)
df_6_prefer["preference"] = (
    df_6_prefer["preference"].astype(str)
    .str.replace("Option ", "", regex=False)
    .astype(int)
)

df_6_prefer = df_6_prefer[["respondent_id", "taskNum", "preference"]]


# 3) Mergen & Y bauen

df_6_stack = df_6_distr.merge(df_6_prefer, on=["respondent_id", "taskNum"], how="left")
df_6_stack["Y"] = (df_6_stack["optionNum"] == df_6_stack["preference"]).astype(int)


In [170]:
# surveys
survey_dict = {"S1": "../data/Survey_Adaptation Natural Hazards_First Wave_raw data.csv", "S2": "../data/Survey_Adaptation Natural Hazards_Second Wave_raw data.csv"}
# survey_dict = {"Survey 2": "../data/Survey_Adaptation Natural Hazards_Second Wave_raw data.csv"}
# survey_dict = {"Survey 1": "../data/Survey_Adaptation Natural Hazards_First Wave_raw data.csv"}


df_dict ={}

for s, file in survey_dict.items():
    df = pd.read_csv(file, dtype={"id": "string"}, skiprows=[1,2])
    # remove lines and replace empty cells and rename columnames
    df = df.replace("", pd.NA)
    df.columns = df.columns.str.replace(".", "", regex=False)
    df.columns = df.columns.str.replace(r"\s+", "", regex=True)
    df = dp.rename_columns(df, 'municipality', 'benefits')
    df = dp.to_num_col(df, ['id', 'm'])
    df = df.add_prefix(f'{s}_')
    df_dict[s] = df
    print(len(df))    


1528
774


In [163]:
# merge both surveys and check for uniqueness
keys = pd.read_csv("../data/id_list.csv", sep=';',dtype={"NEW IDS": "Int64", "OLD IDS": "Int64"})
print(keys.duplicated().any())
print(df_dict['S1']['S1_id'].duplicated().any())
print(keys['OLD IDS'].isin(df_dict['S1']['S1_id']).any())

# keys['OLD IDS'] = pd.to_numeric(keys['OLD IDS'])
# keys['NEW IDS'] = pd.to_numeric(keys['NEW IDS'])



merged_waves_df = (
    keys
    .merge(df_dict['S1'], how="inner", left_on="NEW IDS", right_on="S1_id")
    .merge(df_dict['S2'], how="inner", left_on="OLD IDS", right_on="S2_m")
)

(merged_waves_df)

False
True
False


,NEW IDS,OLD IDS,S1_StartDate,S1_EndDate,S1_Status,S1_IPAddress,S1_Progress,S1_Duration(inseconds),S1_Finished,S1_RecordedDate,...,S2_choice9_costs1,S2_choice9_costs2,S2_choice9_exemptions1,S2_choice9_exemptions2,S2_choice9_benefits1,S2_choice9_benefits2,S2_Q_TerminateFlag,S2_Q_R_Del,S2_screened_out,S2_SelectedLanguage
0,307963907634351,318152052069452,2025-05-08 10:24:25,2025-05-08 10:37:59,IP Address,193.246.191.125,100,814,True,2025-05-08 10:38:00,...,Le persone pagano in proporzione al loro reddito,Tutte le persone pagano lo stesso importo,Le persone a basso e medio reddito sono esenta...,Le persone a basso reddito sono esentate dai c...,Livelli di protezione uguali per tutti i comuni,"Comuni di grande valore culturale, ad esempio ...",Complete,NaN,False,IT
1,307988881480844,318078313779222,2025-05-08 15:59:17,2025-05-08 16:13:18,IP Address,188.60.117.93,100,840,True,2025-05-08 16:13:18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FR
2,307988881480844,318078313779222,2025-05-08 15:59:17,2025-05-08 16:13:18,IP Address,188.60.117.93,100,840,True,2025-05-08 16:13:18,...,Tous les citoyens paient le même montant,Les personnes paient proportionnellement à leu...,Aucun groupe n'est exempté des coûts,Les personnes à faible revenu peuvent être exe...,Les municipalités les plus touchées par les ri...,Les municipalités économiquement prospères,Complete,NaN,False,FR
3,307799977910308,318078398527710,2025-05-06 16:55:34,2025-05-06 17:09:39,IP Address,194.209.36.231,100,844,True,2025-05-06 17:09:40,...,Menschen zahlen proportional zu ihrem Einkommen,Unternehmen zahlen proportional zu ihrem CO2-A...,Keine Gruppen sind von den Kosten ausgenommen,Keine Gruppen sind von den Kosten ausgenommen,"Gemeinden, in denen Menschen seit vielen Jahre...",Gemeinden mit vielen Kulturgütern wie z.B. his...,Complete,NaN,False,DE
4,307988976305297,318078398527711,2025-05-08 15:57:09,2025-05-08 16:15:22,IP Address,146.185.81.236,100,1092,True,2025-05-08 16:15:22,...,Alle Menschen zahlen den gleichen Betrag,Unternehmen zahlen proportional zu ihrem CO2-A...,mit Ausnahme von Menschen mit niedrigem und mi...,Keine Gruppen sind von den Kosten ausgenommen,"Gemeinden, in denen Menschen seit vielen Jahre...","Gemeinden, in denen Menschen seit vielen Jahre...",Complete,NaN,False,DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,307714474953952,318152004294718,2025-05-05 16:43:24,2025-05-05 16:57:17,IP Address,158.181.109.181,100,832,True,2025-05-05 16:57:17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE
657,307714474953952,318152004294718,2025-05-05 16:43:24,2025-05-05 16:57:17,IP Address,158.181.109.181,100,832,True,2025-05-05 16:57:17,...,Alle Menschen zahlen den gleichen Betrag,Menschen zahlen proportional zu ihrem Einkommen,mit Ausnahme von Menschen mit niedrigem und mi...,Keine Gruppen sind von den Kosten ausgenommen,"Gemeinden, in denen Menschen seit vielen Jahre...","Gemeinden, in denen Menschen seit vielen Jahre...",Complete,NaN,False,DE
658,307714474969321,318152004294719,2025-05-05 13:41:59,2025-05-05 13:54:37,IP Address,185.68.67.90,100,757,True,2025-05-05 13:54:37,...,Alle Menschen zahlen den gleichen Betrag,"Menschen und Unternehmen, die von Schutzmaßnah...",Keine Gruppen sind von den Kosten ausgenommen,mit Ausnahme von Menschen mit niedrigem Einkommen,"Gemeinden, die am stärksten von Naturgefahren ...","Gemeinden, in denen Menschen seit vielen Jahre...",Complete,NaN,False,DE
659,307714475005954,318152004294722,2025-05-05 15:36:33,2025-05-05 15:54:43,IP Address,45.12.222.43,100,1090,True,2025-05-05 15:54:44,...,Alle Menschen zahlen den gleichen Betrag,Unternehmen zahlen proportional zu ihrem CO2-A...,Keine Gruppen sind von den Kosten ausgenommen,Keine Gruppen sind von den Kosten ausgenommen,"Gemeinden, die am stärksten von Naturgefahren ...",Wirtschaftlich wohlhabende Gemeinden,Complete,NaN,False,DE


In [177]:

def rm_bad_quality(df, survey):
    # Filter: incompletes, screened out, poor quality, quota met
    df[f"{survey}_Finished"] = df[f"{survey}_Finished"].map({"True": True, "False": False})
    df = df[(df[f"{survey}_DistributionChannel"] != "preview") & (df[f"{survey}_Finished"] != False)].copy()
    df = df[~df[f"{survey}_Q_TerminateFlag"].isin(["PoorQuality", "NA", "QuotaMet", "Screened"])].copy()
    return df

merged_waves_df = rm_bad_quality(merged_waves_df, 'S1')
merged_waves_df = rm_bad_quality(merged_waves_df, 'S2')
len(merged_waves_df)

# df[f"S2_Finished"] = df[f"S2_Finished"].map({"True": True, "False": False})
df = df_dict['S2']
print(df["S2_Finished"] != False)
df = df_dict['S1']
print(df["S1_Finished"] != False)


0       True
1       True
2       True
3       True
4       True
       ...  
769     True
770    False
771    False
772    False
773    False
Name: S2_Finished, Length: 774, dtype: bool
0        True
1        True
2        True
3        True
4        True
        ...  
1523     True
1524    False
1525    False
1526    False
1527    False
Name: S1_Finished, Length: 1528, dtype: bool


In [ ]:
# 2. Transform likert-scales to numerical values

# Only select named columns and map correct likert number in case for string reply
def transform_likert(df, survey):
    valid_columns_s1 = [f'{s}_{c}' for c in valid_columns_s1 if c in df.columns]

    for c in valid_columns_s1:
        s = df[c].replace(likert_map)
        df[c] = s.astype("string").str.extract(r"(-?\d+\.?\d*)", expand=False)
    dp.to_numeric_col(df, valid_columns_s1)
    return df

df = transform_likert(df, 'S1')
df = transform_likert(df, 'S2')

df[valid_columns_s1]


,1_conjoint_acceptance_2,own_municipality_1,sensitivity_nh_1,climatechange_nh_1,sensitivity_nh_3,lreco_2,5_conjoint_acceptance_2,5_conjoint_acceptance_1,2_conjoint_acceptance_2,4_conjoint_acceptance_2,...,own_municipality_3,6_conjoint_acceptance_2,2_conjoint_acceptance_1,sensitivity_nh_4,own_municipality_2,env_values_3,climatechange_nh_3,sensitivity_nh_2,lreco_1,4_conjoint_acceptance_1
3,4.0,4.0,3.0,4.0,4.0,4.0,5.0,5.0,5.0,4.0,...,5.0,4.0,5.0,5.0,5.0,5.0,5.0,2.0,3.0,5.0
4,4.0,4.0,3.0,1.0,3.0,4.0,3.0,3.0,3.0,3.0,...,4.0,3.0,3.0,4.0,3.0,3.0,5.0,4.0,3.0,3.0
5,4.0,NaN,1.0,1.0,1.0,4.0,3.0,3.0,3.0,4.0,...,5.0,1.0,3.0,6.0,NaN,1.0,6.0,1.0,4.0,3.0
6,1.0,4.0,2.0,4.0,4.0,6.0,3.0,1.0,3.0,1.0,...,4.0,2.0,1.0,4.0,5.0,5.0,3.0,2.0,2.0,3.0
7,1.0,4.0,2.0,3.0,2.0,3.0,2.0,2.0,3.0,2.0,...,4.0,2.0,2.0,5.0,4.0,1.0,5.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1517,3.0,4.0,NaN,NaN,3.0,5.0,3.0,4.0,4.0,3.0,...,2.0,2.0,3.0,5.0,4.0,1.0,6.0,NaN,NaN,3.0
1518,4.0,4.0,6.0,6.0,6.0,4.0,4.0,5.0,5.0,3.0,...,2.0,5.0,4.0,6.0,2.0,1.0,6.0,6.0,4.0,4.0
1519,1.0,6.0,6.0,6.0,6.0,6.0,1.0,6.0,6.0,6.0,...,6.0,6.0,1.0,6.0,6.0,1.0,6.0,6.0,1.0,1.0
1524,3.0,4.0,1.0,4.0,4.0,6.0,3.0,4.0,1.0,1.0,...,6.0,1.0,3.0,6.0,3.0,1.0,6.0,3.0,1.0,3.0


In [49]:
df["Finished"]

3       True
4       True
5       True
6       True
7       True
        ... 
1517    True
1518    True
1519    True
1524    True
1525    True
Name: Finished, Length: 1173, dtype: bool

In [ ]:
# TODO: check with others if this filter is not too loose
# remove speeding and very slow respondents
# Umbenennen von Duration(inseconds) -> duration (nach dem Spalten-Cleaning)
def rm_speeders(df):
    if "Duration(inseconds)" in df.columns:
        df = dp.rename_columns(df, "Duration(inseconds)", "duration")

        # Sicherstellen, dass duration numerisch ist
        df["S1_duration"] = pd.to_numeric(df["S1_duration"], errors="coerce")
        df["S2_duration"] = pd.to_numeric(df["S2_duration"], errors="coerce")

        p5s1 = df["S1_duration"].quantile(0.05)
        p5s2 = df["S2_duration"].quantile(0.05)
        p95s1 = df["S1_duration"].quantile(0.95)
        p95s2 = df["S2_duration"].quantile(0.95)

        df_timed_replies = df[(df["S1_duration"] > p5s1) & (df["S2_duration"] < p95s2) & (df["S1_duration"] < p95s1) & (df["S2_duration"] < p95s2)].copy()
        
        print(f'{len(df)-len(df_timed_replies)} respondents were faster than ({p5s1}|{p5s2})s or slower than ({p95s1}|{p95s2})s')

    else:
        print("Duration filter already done")
    return df

df = rm_speeders(df)

len(df)




1055

In [51]:
# remove straightliners
# (respondents who gave the same answer across a set of Likert-scale questions)
def rm_straightliners(df):
    # check 
    likert_costs_cols = [c for c in df.columns if c.startswith("likert_costs_")]
    identity_cols = [c for c in df.columns if c.startswith("identity_group")]
    gal_tan_cols = [c for c in df.columns if c.startswith("gal_tan_")]
    deservingness_cols = [c for c in df.columns if c.startswith("deservingness")]

    straightliners_costs = df[likert_costs_cols].nunique(axis=1, dropna=True).eq(1)
    straightliners_identity = df[identity_cols].nunique(axis=1, dropna=True).eq(1)
    straightliners_gal_tan = df[gal_tan_cols].nunique(axis=1, dropna=True).eq(1)
    straightliners_deservingness = df[deservingness_cols].nunique(axis=1, dropna=True).eq(1)

    straightliners = straightliners_costs | straightliners_identity | straightliners_gal_tan | straightliners_deservingness
    df_cleaned = df.loc[~straightliners].copy()
df_cleaned = rm_straightliners(df)
len(df_cleaned)

1055

In [52]:
# filter identical IP addresses
# IPs mit mehr als 2 verschiedenen ids
def filter_double_ipa(df):
    shared_ips = (
        df.groupby("IPAddress")["id"]
        .nunique()
        .reset_index(name="user_count")
    )
    shared_ips = shared_ips[shared_ips["user_count"] > 2]["IPAddress"]

    # Nutzer mit diesen IPs
    users_to_remove = df[df["IPAddress"].isin(shared_ips)].copy()

    # anti_join nach id & IPAddress
    df = df.merge(
        users_to_remove[["id", "IPAddress"]].drop_duplicates(),
        on=["id", "IPAddress"],
        how="left",
        indicator=True,
    )
    df = df[df["_merge"] == "left_only"].drop(columns="_merge").copy()
    return df


932

In [53]:
# remove double ipas
df_cleaned = filter_double_ipa(df_cleaned)
len(df_cleaned)

# reset unique ids
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned["respondent_id"] = df_cleaned.index + 1
len(df_cleaned)

932

In [61]:
def demographics_mapping(df, mapping_dict, column_patterns=None):
    df = df.copy()

    if column_patterns:
        regex = re.compile("|".join(column_patterns))
        cols = [c for c in df.columns if regex.search(c)]
    else:
        cols = df.columns.tolist()

    df[cols] = df[cols].astype("string").apply(lambda s: s.str.strip())
    df[cols] = df[cols].replace(mapping_dict)

    return df

df_cleaned = demographics_mapping(df_cleaned, demographics_dict, column_patterns=[r"^gender$", r"^age$", r"^education$", r"^income$", r"^language$", r"^language_region$", r"^party_choice$"])


{'Female': 'Female', 'Male': 'Male', 'Non-binary / Other': nan, 'Prefer not to say ': nan, '18 - 34': '18 - 34', '35 - 49': '35 - 49', '50 or older': '50+', 'No high school diploma': 'Below Secondary', 'Vocational training or apprenticeship': 'Vocational training or apprenticeship', 'High school diploma': 'High school diploma', "Bachelor's degree": 'University degree', "Master's degree": 'University degree', 'Doctoral or professional degree (e.g. PhD, MD, JD)': 'University degree', 'Prefer not to say': nan, 'Less than CHF 50,000': 'Low', 'CHF 50,000 - CHF 70,000': 'Low', 'CHF 70,000 - CHF 100,000': 'Mid', 'CHF 100,001 - CHF 150,000': 'Mid', 'CHF 150,001 - CHF 250,000': 'High', 'More than CHF 250,000': 'High', "I don't know": nan, 'Deutsch': 'German', 'Français': 'French', 'English': 'English', 'Italiano': 'Italian', 'German-speaking region': 'German-speaking region', 'Italian-speaking region': 'Italian-speaking region', 'French-speaking region': 'French-speaking region', 'Romansh-speak

In [66]:
# df["education"].astype("string").str.strip().apply(lambda s: s.map(demographics_dict))
df_cleaned["age"]



0          50+
1          50+
2          50+
3          50+
4      35 - 49
        ...   
927    18 - 34
928    18 - 34
929    18 - 34
930    18 - 34
931    18 - 34
Name: age, Length: 932, dtype: string